# Analisi Esplorativa dei Dati (EDA) - Template

Questo notebook serve come template di base per eseguire un'analisi esplorativa (EDA) su un nuovo dataset.

**Obiettivi dell'EDA:**
1.  Comprendere la struttura del dataset.
2.  Identificare dati mancanti, duplicati o anomali.
3.  Analizzare la distribuzione delle singole variabili (Analisi Univariata).
4.  Esplorare le relazioni tra le variabili (Analisi Bivariata/Multivariata).
5.  Ricavare insight iniziali e formulare ipotesi.

## 1. Setup: Importare le Librerie

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Impostazioni per una migliore visualizzazione
%matplotlib inline
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## 2. Caricamento Dati

In [ ]:
# Sostituisci 'IL_TUO_DATASET.csv' con il percorso al tuo file
# Se hai un Excel, usa pd.read_excel()
try:
    file_path = 'IL_TUO_DATASET.csv' 
    df = pd.read_csv(file_path)
except FileNotFoundError:
    print("File non trovato. Assicurati che il percorso sia corretto.")
    # Come fallback, creo un dataset di esempio per far funzionare il resto del notebook
    data = {
        'eta': [25, 30, np.nan, 45, 30, 50, 22, 35, 40, 28],
        'genere': ['M', 'F', 'M', 'F', 'M', 'F', 'M', 'F', 'M', 'F'],
        'punteggio': [85, 90, 78, 92, 78, 88, 76, 95, 82, 89],
        'citta': ['Roma', 'Milano', 'Roma', 'Napoli', 'Torino', 'Milano', 'Roma', 'Napoli', 'Milano', 'Torino'],
        'target': [1, 0, 0, 1, 0, 1, 0, 1, 1, 0]
    }
    df = pd.DataFrame(data)
    print("\n--- Creato dataset di esempio --- ")

## 3. Ispezione Iniziale (Visione d'Insieme)

### 3.1 Dimensioni del Dataset

In [ ]:
print(f"Il dataset ha {df.shape[0]} righe e {df.shape[1]} colonne.")

### 3.2 Anteprima dei Dati

In [ ]:
# Visualizza le prime 5 righe
df.head()

In [ ]:
# Visualizza le ultime 5 righe
df.tail()

### 3.3 Tipi di Dati e Valori Mancanti (Sintesi)

In [ ]:
# Ottiene informazioni sui tipi di colonna (Dtypes) e sui valori non nulli
df.info()

### 3.4 Statistiche Descrittive

In [ ]:
# Statistiche descrittive per le colonne numeriche
# (conteggio, media, deviazione standard, min, max, percentili)
df.describe()

In [ ]:
# Statistiche descrittive per le colonne categoriche (tipo 'object')
# (conteggio, valori unici, valore più frequente, frequenza)
df.describe(include=['object'])

## 4. Pulizia Dati (Gestione Missing e Duplicati)

### 4.1 Valori Mancanti (NaN)

In [ ]:
# Controlla il numero (e la percentuale) di valori mancanti per colonna
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100

pd.DataFrame({'Conteggio Mancanti': missing_values, 'Percentuale (%)': missing_percent}).sort_values(by='Percentuale (%)', ascending=False)

**Strategie (da affinare):**
* **Drop:** Se i valori mancanti sono pochi (es. <5%) o se una colonna è quasi tutta vuota, si può eliminare (`df.dropna()`).
* **Imputazione (Numerici):** Sostituire con media, mediana (`df['colonna'].fillna(df['colonna'].median())`).
* **Imputazione (Categorici):** Sostituire con la moda (valore più frequente) (`df['colonna'].fillna(df['colonna'].mode()[0])`).

### 4.2 Dati Duplicati

In [ ]:
num_duplicati = df.duplicated().sum()
print(f"Ci sono {num_duplicati} righe duplicate nel dataset.")

In [ ]:
# Esempio per rimuovere duplicati (da decommentare se necessario)
# df_pulito = df.drop_duplicates()
# print(f"Dimensioni dopo rimozione duplicati: {df_pulito.shape}")

## 5. Analisi Univariata (Analisi delle singole variabili)

Separiamo le colonne numeriche da quelle categoriche per analizzarle con i grafici appropriati.

In [ ]:
colonne_numeriche = df.select_dtypes(include=np.number).columns
colonne_categoriche = df.select_dtypes(include='object').columns

print("Colonne Numeriche:", list(colonne_numeriche))
print("Colonne Categoriche:", list(colonne_categoriche))

### 5.1 Analisi Variabili Numeriche

In [ ]:
# Plot della distribuzione (Istogramma + KDE) per ogni variabile numerica
for col in colonne_numeriche:
    plt.figure(figsize=(10, 5))
    sns.histplot(df[col], kde=True, bins=30)
    plt.title(f'Distribuzione di {col}')
    plt.xlabel(col)
    plt.ylabel('Frequenza')
    plt.show()

In [ ]:
# Boxplot per identificare gli outlier
for col in colonne_numeriche:
    plt.figure(figsize=(10, 3))
    sns.boxplot(x=df[col])
    plt.title(f'Boxplot di {col}')
    plt.xlabel(col)
    plt.show()

### 5.2 Analisi Variabili Categoriche

In [ ]:
# Countplot (grafico a barre) per ogni variabile categorica
for col in colonne_categoriche:
    plt.figure(figsize=(10, 5))
    # Usiamo 'y' se ci sono molte categorie o nomi lunghi
    sns.countplot(y=col, data=df, order=df[col].value_counts().index)
    plt.title(f'Conteggio di {col}')
    plt.xlabel('Conteggio')
    plt.ylabel(col)
    plt.show()

## 6. Analisi Bivariata e Multivariata (Relazioni)

### 6.1 Matrice di Correlazione (Numerico vs Numerico)

In [ ]:
# Calcoliamo la matrice di correlazione (di Pearson)
corr_matrix = df[colonne_numeriche].corr()

# Visualizziamo la heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Matrice di Correlazione Heatmap')
plt.show()

### 6.2 Pairplot (Visione d'insieme)

In [ ]:
# Scatter plot per ogni coppia di variabili numeriche
# Sostituisci 'NOME_COLONNA_TARGET' (es. 'genere' o 'target') per colorare i punti
# Se non hai una colonna target chiara, rimuovi 'hue=...'

try:
    # Prova a usare una colonna categorica o la target per 'hue'
    hue_col = 'target' if 'target' in df.columns else (colonne_categoriche[0] if len(colonne_categoriche) > 0 else None)
    sns.pairplot(df, hue=hue_col, vars=colonne_numeriche, diag_kind='kde')
except Exception as e:
    print(f"Errore nel Pairplot (forse non ci sono colonne numeriche?): {e}")
    # Fallback senza hue se la colonna dà problemi
    if 'colonne_numeriche' in locals() and len(colonne_numeriche) > 0:
       sns.pairplot(df, vars=colonne_numeriche, diag_kind='kde')

plt.suptitle('Pairplot delle variabili numeriche', y=1.02)
plt.show()

### 6.3 Analisi Categoria vs Numero

In [ ]:
# Esempio: Boxplot raggruppati
# Come varia una variabile numerica (es. 'punteggio') al variare di una categorica (es. 'citta')?

# Sostituisci 'colonna_categorica_esempio' e 'colonna_numerica_esempio' con i nomi delle tue colonne
colonna_categorica_esempio = 'citta' 
colonna_numerica_esempio = 'punteggio'

if colonna_categorica_esempio in df.columns and colonna_numerica_esempio in df.columns:
    plt.figure(figsize=(12, 7))
    sns.boxplot(x=colonna_categorica_esempio, y=colonna_numerica_esempio, data=df)
    plt.title(f'{colonna_numerica_esempio} vs {colonna_categorica_esempio}')
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
# Esempio: Violin plot (simile al boxplot ma mostra anche la distribuzione)
if colonna_categorica_esempio in df.columns and colonna_numerica_esempio in df.columns:
    plt.figure(figsize=(12, 7))
    sns.violinplot(x=colonna_categorica_esempio, y=colonna_numerica_esempio, data=df)
    plt.title(f'Violin Plot: {colonna_numerica_esempio} vs {colonna_categorica_esempio}')
    plt.xticks(rotation=45)
    plt.show()

## 7. Conclusioni Iniziali e Prossimi Passi

*(Questa sezione è per le tue note)*

**Riassumi qui le tue scoperte:**

* **Qualità dei Dati:** (Es: "La colonna 'età' ha il 10% di valori mancanti, che dovranno essere imputati con la mediana.")
* **Distribuzioni:** (Es: "La variabile 'punteggio' segue una distribuzione approssimativamente normale.", "La 'città' più frequente è Roma.")
* **Outlier:** (Es: "Sono presenti alcuni outlier significativi nel 'punteggio' che potrebbero richiedere un'indagine.")
* **Correlazioni:** (Es: "C'è una forte correlazione positiva tra 'punteggio' e 'target' (0.75).")
* **Relazioni:** (Es: "Gli utenti di Milano sembrano avere un 'punteggio' mediano più alto.")

**Prossimi Passi (Feature Engineering):**

* (Es: "Trasformare la colonna 'citta' in variabili dummy (One-Hot Encoding).")
* (Es: "Creare una nuova feature 'fascia_eta' raggruppando la colonna 'eta'.")
* (Es: "Normalizzare/Standardizzare le feature numeriche prima del modeling.")